In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle

# How stocks react when Covid just break out (From google trends, the topic reached 100% popularity on March 22, which is a Sunday. So we choose March 23, Monday for stock price check)

In [3]:
spy = pd.read_csv('SPY_price.csv')
spy['return'] = spy['close'].pct_change()
spy['data_date'] = pd.to_datetime(spy['data_date'])

In [6]:
spy_ret = spy[['data_date', 'return']].dropna().sort_values('data_date').set_index('data_date')

In [8]:
covid_trend = pickle.load(open('../covid_trends.pkl', 'rb'))
covid_trend_df = pd.DataFrame(covid_trend['trends_interest_over_time']['data'])
covid_trend_df.shape

(51, 3)

In [9]:
covid_trend_df = pd.DataFrame(covid_trend['trends_interest_over_time']['data'])
covid_trend_df['has_values'] = covid_trend_df['values'].apply(lambda x: x[0]['has_value'])
covid_trend_df = covid_trend_df[covid_trend_df['has_values']==True]
covid_trend_df['values2'] = covid_trend_df['values'].apply(lambda x: x[0]['value'])
covid_trend_df['date'] = covid_trend_df['date_utc'].apply(lambda x: x.split('T')[0])
covid_trend_df['date'] = pd.to_datetime(covid_trend_df['date'])

In [13]:
covid_search = covid_trend_df[['date', 'values2']].set_index('date')

In [15]:
wkly_return = spy_ret.resample('W').sum()
wkly_search = covid_search.resample('W').sum()

In [17]:
spy_covid = pd.concat([wkly_return, wkly_search], axis=1)
spy_covid.columns = ['spy_return%', 'COVID_search']
spy_covid['spy_return%'] = spy_covid['spy_return%']*100

In [22]:
spy_covid = spy_covid.dropna()

In [23]:
spy_covid.corr()

,spy_return%,COVID_search
spy_return%,1.000000,-0.202749
COVID_search,-0.202749,1.000000


### After March correlation is even larger

In [25]:
spy_covid['2020-03':].corr()

,spy_return%,COVID_search
spy_return%,1.000000,-0.278849
COVID_search,-0.278849,1.000000


In [28]:
spy_covid.head()

,spy_return%,COVID_search
data_date,,
2020-01-26,2.399714,0.0
2020-02-02,-0.165872,0.0
2020-02-09,-3.183401,1.0
2020-02-16,-0.607931,1.0
2020-02-23,4.407522,4.0


In [ ]:
spy_covid.to_csv('')

# Plotly

In [43]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [60]:
news_count = pd.read_csv('NYTnews_keyword_monthly.csv', index_col=0)
news_count['date'] = '2020-'+news_count['month'].astype(str)+'-15'
news_count['date'] = pd.to_datetime(news_count['date'])

In [62]:
news_count

,month,covid_kw_count,mention_per_news,date
0,1,18,0.402,2020-01-15
1,2,89,2.099,2020-02-15
2,3,373,7.639,2020-03-15
3,4,407,8.109,2020-04-15
4,5,348,8.006,2020-05-15
5,6,238,5.298,2020-06-15
6,7,240,5.382,2020-07-15
7,8,221,4.979,2020-08-15
8,9,223,4.838,2020-09-15
9,10,337,6.411,2020-10-15


In [70]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Line(x=spy_covid.index, y=spy_covid['spy_return%'], name="spy return%"),
    secondary_y=False,
)

fig.add_trace(
    go.Line(x=spy_covid.index, y=spy_covid['COVID_search'], name="covid search"),
    secondary_y=True,
)

fig.add_trace(
    go.Bar(x=news_count.date, y=news_count['mention_per_news'], 
           name="NYTnews", opacity=0.3),
    secondary_y=True
)

fig.update_layout(
    title_text="SPY Return vs COVID Search Trends in 2020"
)

fig.update_yaxes(title_text="SPY Return%", secondary_y=False, showgrid=False)
fig.update_yaxes(title_text="Covid Search Trends", secondary_y=True, showgrid=False)

fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.show()